In [1]:
import pandas as pd

In [2]:
from google.colab import auth
auth.authenticate_user()
print('Authenticated')

Authenticated


In [3]:
project_id = 'novo-udemy-395521'

In [33]:
from google.cloud import bigquery

cliente = bigquery.Client(project=project_id)

## 6 - Quantos chamados com o subtipo "Perturbação do sossego" foram abertos desde 01/01/2022 até 31/12/2023 (incluindo extremidades)?

In [34]:
consulta_1 = """
SELECT COUNT(*) AS total_chamados
FROM `datario.administracao_servicos_publicos.chamado_1746`
WHERE subtipo = 'Perturbação do sossego'
AND data_inicio BETWEEN '2022-01-01' AND '2023-12-31'
"""
resultado_1 = cliente.query(consulta_1).result()
for linha in resultado_1:
    print(f"Total de chamados: {linha['total_chamados']}")

Total de chamados: 42408


## 7 - Selecione os chamados com esse subtipo que foram abertos durante os eventos contidos na tabela de eventos (Reveillon, Carnaval e Rock in Rio).

In [35]:
query = """
WITH eventos AS (
  SELECT data_inicial, data_final, evento
  FROM `datario.turismo_fluxo_visitantes.rede_hoteleira_ocupacao_eventos`
)
SELECT c.id_chamado, c.data_inicio, e.evento
FROM `datario.administracao_servicos_publicos.chamado_1746` c
JOIN eventos e
ON c.data_inicio BETWEEN e.data_inicial AND e.data_final
WHERE c.subtipo = 'Perturbação do sossego'
"""

tarefa_query = cliente.query(query)
resultados = tarefa_query.result()

for linha in resultados:
    print(f"ID Chamado: {linha.id_chamado}, Data Início: {linha.data_inicio}, Evento: {linha.evento}")


ID Chamado: 18331532, Data Início: 2023-02-20 08:58:36, Evento: Carnaval
ID Chamado: 18333126, Data Início: 2023-02-20 22:22:35, Evento: Carnaval
ID Chamado: 18329659, Data Início: 2023-02-18 23:39:48, Evento: Carnaval
ID Chamado: 18329561, Data Início: 2023-02-18 21:57:42, Evento: Carnaval
ID Chamado: 18331149, Data Início: 2023-02-19 21:36:51, Evento: Carnaval
ID Chamado: 17679482, Data Início: 2022-09-09 10:06:49, Evento: Rock in Rio
ID Chamado: 17659324, Data Início: 2022-09-02 10:51:01, Evento: Rock in Rio
ID Chamado: 17678519, Data Início: 2022-09-08 22:52:52, Evento: Rock in Rio
ID Chamado: 17676859, Data Início: 2022-09-08 14:09:44, Evento: Rock in Rio
ID Chamado: 17684217, Data Início: 2022-09-10 20:51:05, Evento: Rock in Rio
ID Chamado: 17662646, Data Início: 2022-09-03 13:05:15, Evento: Rock in Rio
ID Chamado: 17684011, Data Início: 2022-09-10 18:35:35, Evento: Rock in Rio
ID Chamado: 17682424, Data Início: 2022-09-09 21:23:40, Evento: Rock in Rio
ID Chamado: 17661695, Data 

## 8 - Quantos chamados desse subtipo foram abertos em cada evento?


In [36]:
query = """
WITH eventos AS (
  SELECT data_inicial, data_final, evento
  FROM `datario.turismo_fluxo_visitantes.rede_hoteleira_ocupacao_eventos`
)
SELECT e.evento, COUNT(*) AS total_chamados
FROM `datario.administracao_servicos_publicos.chamado_1746` c
JOIN eventos e
ON c.data_inicio BETWEEN e.data_inicial AND e.data_final
WHERE c.subtipo = 'Perturbação do sossego'
GROUP BY e.evento
"""

tarefa_query = cliente.query(query)
resultados = tarefa_query.result()

for linha in resultados:
    print(f"Evento: {linha.evento}, Total de chamados: {linha.total_chamados}")


Evento: Reveillon, Total de chamados: 79
Evento: Carnaval, Total de chamados: 197
Evento: Rock in Rio, Total de chamados: 518


## 9 - Qual evento teve a maior média diária de chamados abertos desse subtipo?

In [38]:
query = """
WITH eventos AS (
  SELECT data_inicial, data_final, evento, DATE_DIFF(data_final, data_inicial, DAY) + 1 AS duracao_dias
  FROM `datario.turismo_fluxo_visitantes.rede_hoteleira_ocupacao_eventos`
)
, chamados_evento AS (
  SELECT e.evento, COUNT(*) AS total_chamados
  FROM `datario.administracao_servicos_publicos.chamado_1746` c
  JOIN eventos e
  ON c.data_inicio BETWEEN e.data_inicial AND e.data_final
  WHERE c.subtipo = 'Perturbação do sossego'
  GROUP BY e.evento
)
SELECT evento, total_chamados / duracao_dias AS media_diaria
FROM chamados_evento
JOIN eventos USING(evento)
ORDER BY media_diaria DESC
LIMIT 1
"""

tarefa_query = cliente.query(query)
resultados = tarefa_query.result()

for linha in resultados:
    print(f"Evento com maior média diária: {linha.evento},  Média diária: {linha.media_diaria}")


Evento com maior média diária: Rock in Rio, / Média diária: 172.66666666666666


## 10 - Compare as médias diárias de chamados abertos desse subtipo durante os eventos específicos (Reveillon, Carnaval e Rock in Rio) e a média diária de chamados abertos desse subtipo considerando todo o período de 01/01/2022 até 31/12/2023.


In [39]:
query_periodo_total = """
SELECT
  COUNT(*) AS total_chamados,
  DATE_DIFF('2023-12-31', '2022-01-01', DAY) + 1 AS total_dias
FROM `datario.administracao_servicos_publicos.chamado_1746`
WHERE subtipo = 'Perturbação do sossego'
AND data_inicio BETWEEN '2022-01-01' AND '2023-12-31'
"""

tarefa_query_total = cliente.query(query_periodo_total)
resultados_total = tarefa_query_total.result()

for linha in resultados_total:
    media_diaria_total = linha.total_chamados / linha.total_dias
    print(f"Média diária de chamados no período total: {media_diaria_total}")


Média diária de chamados no período total: 58.09315068493151


In [40]:
query_eventos = """
WITH eventos AS (
  SELECT data_inicial, data_final, evento, DATE_DIFF(data_final, data_inicial, DAY) + 1 AS duracao_dias
  FROM `datario.turismo_fluxo_visitantes.rede_hoteleira_ocupacao_eventos`
)
, chamados_evento AS (
  SELECT e.evento, COUNT(*) AS total_chamados
  FROM `datario.administracao_servicos_publicos.chamado_1746` c
  JOIN eventos e
  ON c.data_inicio BETWEEN e.data_inicial AND e.data_final
  WHERE c.subtipo = 'Perturbação do sossego'
  GROUP BY e.evento
)
SELECT evento, total_chamados, duracao_dias, (total_chamados / duracao_dias) AS media_diaria
FROM chamados_evento
JOIN eventos USING(evento)
"""

tarefa_query_eventos = cliente.query(query_eventos)
resultados_eventos = tarefa_query_eventos.result()

for linha in resultados_eventos:
    print(f"Evento: {linha.evento}, Média diária de chamados: {linha.media_diaria}")


Evento: Carnaval, Média diária de chamados: 49.25
Evento: Rock in Rio, Média diária de chamados: 172.66666666666666
Evento: Rock in Rio, Média diária de chamados: 129.5
Evento: Reveillon, Média diária de chamados: 26.333333333333332
